In [8]:
!pip install graphviz


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: C:\Users\Yasin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import requests
import json
import re
from IPython.display import display, Markdown

OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL = "phi3:mini"

def ask_llm(prompt):
    response = requests.post(
        OLLAMA_URL,
        json={
            "model": MODEL,
            "prompt": prompt,
            "stream": False
        }
    )
    return response.json()["response"]


In [2]:
def extract_json(text):
    text = text.strip()
    json_match = re.search(r'\{.*\}', text, re.DOTALL)

    if not json_match:
        raise ValueError("No JSON found in LLM output")

    json_text = json_match.group()

    return json.loads(json_text)


In [3]:
def generate_adaptive_roadmap(course, level):
    prompt = f"""
You are a senior curriculum architect.

Generate a COMPLETE adaptive roadmap for {course}.

User Level: {level}

Adaptation Rules:

If Beginner:
- Include full fundamentals
- Slow progression
- Concept clarity focus
- More foundational branches

If Intermediate:
- Assume basic knowledge
- Skip very basic HTML/CSS intro
- Add practical projects
- Add architecture concepts

If Advanced:
- Skip fundamentals
- Focus on system design
- Optimization
- Security
- Scalability
- DevOps
- Performance engineering

Important:
- Topic 1 MUST be Introduction to {course}
- 6–10 main branches
- Each branch must have 4–7 subtopics
- Cover entire ecosystem of the course
- No explanation text
- Only return valid JSON

Return format:

{{
  "title": "{course}",
  "level": "{level}",
  "branches": [
    {{
      "name": "Main Domain",
      "subtopics": [
        "Subtopic 1",
        "Subtopic 2"
      ]
    }}
  ]
}}
"""

    raw = ask_llm(prompt)
    return extract_json(raw)


In [4]:
def generate_nested_mindmap(roadmap):
    mermaid = f"""```mermaid
mindmap
  root(({roadmap['title']} - {roadmap['level']}))
"""

    for branch in roadmap["branches"]:
        mermaid += f"    {branch['name']}\n"
        for sub in branch["subtopics"]:
            mermaid += f"      {sub}\n"

    mermaid += "```"

    display(Markdown(mermaid))


In [ ]:
selected_course = "Web Development"

# This comes from initial MCQ evaluation
user_level = "Beginner"  # Change to Intermediate / Advanced

roadmap = generate_adaptive_roadmap(selected_course, user_level)

generate_nested_mindmap(roadmap)


```mermaid
mindmap
  root((Web Development - Beginner))
    Introduction to Web Development
      HTML Fundamentals
      CSS for Design
    Website Structure and Layout
      Understanding Semantic HTML
      Responsive Navigation Menus
    Introduction to JavaScript Basics
      Variables, Data Types, and Operators
      Control Structures: Loops & Conditionals
    Interactive Web Components with Events
      Handling User Interaction
      Event Delegation Techniques
    Responsive Design Principles
      Flexbox and Grid Systems
      Media Queries for Adaptability
    Creating Accessible Websites
      Semantic HTML5 Elements
      Web Content Accessibility Guidelines (WCAG)
    Optimization Techniques and Best Practices
      Minifying Resources
      Caching for Improved Load Times
```

In [9]:
from graphviz import Digraph

def generate_tree(roadmap):
    dot = Digraph(format="png")
    dot.attr(rankdir='LR', splines='curved', nodesep='0.6', ranksep='1')

    dot.attr('node',
             shape='box',
             style='rounded,filled',
             fillcolor='#2E3B4E',
             fontcolor='white')

    root = roadmap["title"]
    dot.node(root)

    for branch in roadmap["branches"]:
        dot.node(branch["name"])
        dot.edge(root, branch["name"])

        for sub in branch["subtopics"]:
            dot.node(sub)
            dot.edge(branch["name"], sub)

    return dot



In [10]:
roadmap = {
    "title": "Web Development",
    "branches": [
        {
            "name": "Frontend",
            "subtopics": ["HTML", "CSS", "JavaScript"]
        },
        {
            "name": "Backend",
            "subtopics": ["Node.js", "Databases", "APIs"]
        }
    ]
}

generate_tree(roadmap)


ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH